In [1]:
!pip install Pillow
!pip install numpy

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.6 MB 5.6 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.6 MB 3.0 MB/s eta 0:00:01
   -------------------------------- ------- 2.1/2.6 MB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.6 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/12.6 MB 5.0 MB/s eta 0:00:03
   ----------- ---------------------------- 3.7/12.6 MB 6.4 MB/s eta 0:00:02
   ------------------ --------------------- 5.8/12.6 MB 7.5 MB/s eta 0:00:01
   ---------------------------- ----------- 8.9/12.6 MB 8.9 MB/s eta 0:00:01
   --------------------------------- ------ 10.5/12.6 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 9.4 MB/s eta 0:00:00


In [3]:
from PIL import Image, ImageDraw
import numpy as np

def image_to_led_style(img_path, matrix_size):
    # Load the image
    img = Image.open(img_path)
    img = img.convert('RGB')
    
    # Determine the size of each block
    width, height = img.size
    block_width = width // matrix_size[1]
    block_height = height // matrix_size[0]
    
    # Create a new image with the same dimensions
    new_img = Image.new('RGB', (width, height), 'black')  # Start with a black background
    draw = ImageDraw.Draw(new_img)
    
    # Calculate the center of each block for placing the dot
    dot_radius = min(block_width, block_height) // 4  # Radius of the dot
    
    # Iterate over each block
    for i in range(matrix_size[0]):
        for j in range(matrix_size[1]):
            # Compute the center of the block
            center_x = j * block_width + block_width // 2
            center_y = i * block_height + block_height // 2
            
            # Crop the block and calculate the average color
            block = img.crop((j * block_width, i * block_height, (j + 1) * block_width, (i + 1) * block_height))
            mean_color = tuple(np.array(block).mean(axis=(0, 1)).astype(int))
            
            # Draw the dot
            draw.ellipse([(center_x - dot_radius, center_y - dot_radius),
                          (center_x + dot_radius, center_y + dot_radius)],
                         fill=mean_color)
    
    return new_img

# Example usage
matrix_size = (6, 35)  # Define the granularity of the LED matrix
img_path = 'gradient.jpg'
led_img = image_to_led_style(img_path, matrix_size)

# Display or save the LED style image
led_img.show()  # This will display the image
# led_img.save('led_style_output.jpg')  # This will save the image to a file


{0: (np.int64(254), np.int64(59), np.int64(62)), 1: (np.int64(254), np.int64(68), np.int64(114)), 2: (np.int64(254), np.int64(77), np.int64(156)), 3: (np.int64(253), np.int64(85), np.int64(190)), 4: (np.int64(252), np.int64(96), np.int64(232)), 5: (np.int64(244), np.int64(102), np.int64(254)), 6: (np.int64(209), np.int64(105), np.int64(254)), 7: (np.int64(170), np.int64(108), np.int64(254)), 8: (np.int64(137), np.int64(111), np.int64(254)), 9: (np.int64(85), np.int64(116), np.int64(254)), 10: (np.int64(50), np.int64(122), np.int64(254)), 11: (np.int64(45), np.int64(140), np.int64(254)), 12: (np.int64(38), np.int64(160), np.int64(254)), 13: (np.int64(34), np.int64(176), np.int64(254)), 14: (np.int64(26), np.int64(191), np.int64(254)), 15: (np.int64(12), np.int64(212), np.int64(254)), 16: (np.int64(3), np.int64(226), np.int64(253)), 17: (np.int64(35), np.int64(225), np.int64(233)), 18: (np.int64(67), np.int64(219), np.int64(198)), 19: (np.int64(87), np.int64(215), np.int64(169)), 20: (np

In [8]:
from PIL import Image
import numpy as np
import requests
from collections import Counter


rows = 6
cols = 35

# Firebase configuration
firebase_url = "https://allprojects68-default-rtdb.asia-southeast1.firebasedatabase.app/vaiLED/"
color_data_node = "img"  # Set a name for the node where you want to store the data

def upload_color_to_firebase(led_index, rgb_tuple):
    # Store data under vaiLED/img/{led_index}
    url = f"{firebase_url}{color_data_node}/{led_index}.json"
    # Convert NumPy int64 to Python int to avoid serialization issues
    data = [
        int(rgb_tuple[0]),
        int(rgb_tuple[1]),
        int(rgb_tuple[2])
    ]
    try:
        response = requests.put(url, json=data)
        if response.status_code == 200:
            print(f"Successfully updated LED {led_index} with color {rgb_tuple}")
        else:
            print(f"Failed to update LED {led_index}. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error updating LED {led_index} in Firebase: {e}")

def get_dominant_color(image):
    # Convert the image to a list of pixels
    pixels = np.array(image).reshape(-1, 3)
    # Convert each pixel to a tuple (R, G, B)
    pixels = [tuple(pixel) for pixel in pixels]
    # Use Counter to find the most common color
    most_common = Counter(pixels).most_common(1)
    return most_common[0][0]  # Return the most common color

# def divide_image_to_dict_and_upload(img_path, matrix_size):
#     # Load the image
#     img = Image.open(img_path)
#     img = img.convert('RGB')
    
#     # Determine the size of each block
#     width, height = img.size
#     block_width = width // matrix_size[1]
#     block_height = height // matrix_size[0]
    
#     # Counter for dictionary keys
#     counter = 0
    
#     # Iterate over each block
#     for i in range(matrix_size[0]):
#         for j in range(matrix_size[1]):
#             # Compute the bounds of the block
#             left = j * block_width
#             right = (j + 1) * block_width
#             top = i * block_height
#             bottom = (i + 1) * block_height
            
#             # Crop the block and calculate the dominant color
#             block = img.crop((left, top, right, bottom))
#             dominant_color = get_dominant_color(block)
            
#             # Upload the dominant color to Firebase
#             upload_color_to_firebase(counter, dominant_color)
#             counter += 1

def divide_image_to_dict_and_upload(img_path, matrix_size):
    # Load the image
    img = Image.open(img_path)
    img = img.convert('RGB')
    
    # Determine the size of each block
    width, height = img.size
    block_width = width // matrix_size[1]
    block_height = height // matrix_size[0]
    
    # Iterate over each block
    for i in range(matrix_size[0]):
        for j in range(matrix_size[1]):
            # Compute the bounds of the block
            left = j * block_width
            right = (j + 1) * block_width
            top = i * block_height
            bottom = (i + 1) * block_height
            
            # Crop the block and calculate the dominant color
            block = img.crop((left, top, right, bottom))
            dominant_color = get_dominant_color(block)
            
            # Calculate the counter in zigzag pattern
            if i % 2 == 0:
                # Even rows: left to right
                counter = i * matrix_size[1] + j
            else:
                # Odd rows: right to left
                counter = (i + 1) * matrix_size[1] - 1 - j
            
            # Upload the dominant color to Firebase
            upload_color_to_firebase(counter, dominant_color)

# Example usage
matrix_size = (rows, cols)  # Divide the image into a 3x3 matrix
img_path = 'gradient.jpg'  # Path to your image file
divide_image_to_dict_and_upload(img_path, matrix_size)


Successfully updated LED 0 with color (np.uint8(255), np.uint8(57), np.uint8(46))
Successfully updated LED 1 with color (np.uint8(255), np.uint8(66), np.uint8(98))
Successfully updated LED 2 with color (np.uint8(255), np.uint8(79), np.uint8(164))
Successfully updated LED 3 with color (np.uint8(254), np.uint8(88), np.uint8(198))
Successfully updated LED 4 with color (np.uint8(252), np.uint8(100), np.uint8(245))
Successfully updated LED 5 with color (np.uint8(251), np.uint8(102), np.uint8(254))
Successfully updated LED 6 with color (np.uint8(199), np.uint8(106), np.uint8(255))
Successfully updated LED 7 with color (np.uint8(161), np.uint8(109), np.uint8(254))
Successfully updated LED 8 with color (np.uint8(139), np.uint8(112), np.uint8(255))
Successfully updated LED 9 with color (np.uint8(62), np.uint8(118), np.uint8(255))
Successfully updated LED 10 with color (np.uint8(51), np.uint8(121), np.uint8(255))
Successfully updated LED 11 with color (np.uint8(45), np.uint8(142), np.uint8(255))

In [6]:
!pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
